Importation des librairies et du dataset.

In [2]:
import json
import random
import spacy
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import confusion_matrix

# Partie 1 :Netoyage des données.
### Séparation des données et labélisation de celle-ci

In [3]:
rawData = []
with open("./data/image_review_all.json") as fichier:
    for line in fichier:
        dataTempo = json.loads(line)
        rawData.append({
            "id": dataTempo.get('user_id') + "_" + dataTempo.get('business_id'),
            "business_id": dataTempo.get('business_id'),
            "user_id": dataTempo.get('user_id'),
            "rating": dataTempo.get('rating'),
            "review_text": dataTempo.get('review_text'),
            "label": 'positif' if dataTempo.get('rating') > 2 else 'negatif'
        })
rawDF = pd.DataFrame(rawData)

### Séparation des données, pour un réaliser un apprentissage plus pertinent

En effet pour réaliser un apprentissage plus pertinent nous avons fait le choix d'avoir un dataset composé de moitier par des avis positifs et donc l'autre moitier par des avis négatifs

In [4]:
positiveReviewDF = rawDF[rawDF['label'] == 'positif']
negativeReviewDF = rawDF[rawDF['label'] == 'negatif']
positiveReviewDF = positiveReviewDF.sample(frac=1, random_state=42)

Data = pd.concat([negativeReviewDF, positiveReviewDF.head(min(len(negativeReviewDF), len(positiveReviewDF)))])
Data = Data.dropna()
Data = Data.sample(frac=1, random_state=42)

print("Taille du DataFrame = " ,Data.shape)
Data.head()

Taille du DataFrame =  (193308, 6)


,id,business_id,user_id,rating,review_text,label
1459413,110934711920521036628_604088a37cd8bf130361dcf7,604088a37cd8bf130361dcf7,110934711920521036628,1,Why are you guys SO SLOW in the drive thru! Th...,negatif
1055668,116012958129422282207_6046b8efb0e2129e4753558b,6046b8efb0e2129e4753558b,116012958129422282207,5,We found this place from Chet Garners show The...,positif
1408465,108430402497176155594_604172957cd8bf1303625869,604172957cd8bf1303625869,108430402497176155594,5,"Beautiful atmosphere, fantastic food, okay ser...",positif
394571,101062954353697495551_60510eaada79151bfc125676,60510eaada79151bfc125676,101062954353697495551,1,Beef taco was delicious the chicken taco with ...,negatif
734176,109035487472209572473_604b56a677e81aaed3cc93a2,604b56a677e81aaed3cc93a2,109035487472209572473,1,Worst McDonald's I have ever visited,negatif


Sélection de 5 000 données pour notre apprentissage, la tokenisation étant assez longue nous ne pouvons pas prendre le dataset entier.

In [5]:
DataForLearning = Data.head(5000)

Les données étant maintenant nétoyer nous allons pouvoir passer à la partie 2 : Mise en place de la classification avec un méthode vue en cours (deux 
classes à prédire)

# Partie 2 : Mise en place de la classification avec un méthode vue en cours (deux classes à prédire)

Pour notre apprentissage nous utilisons spacy et plus particulièrement son module "en_core_web_sm" car nos avis sont exlusivement en anglais. Pour ce projet la version small du module est suffisant.

In [6]:
nlp = spacy.load("en_core_web_sm")
def tokenisation(text):
  return " ".join([token.lemma_ for token in nlp(text) if not token.is_stop])
DataForLearning["review_text"].apply(tokenisation)

1459413      guy slow drive ! bad location dining area close
1055668    find place Chet Garners daytripper . awesome p...
1408465    beautiful atmosphere , fantastic food , okay s...
394571     beef taco delicious chicken taco soft shell ho...
734176                                  Worst McDonald visit
                                 ...                        
266120     delicious paczki . order online easy . navigat...
620636     way price average food worth wait que .   orde...
435578     time , german restaurant matter . order pretze...
1429300    excellent CUISINE & AFFORDABLE & wide selectio...
1273015    savannah beer good , shrimp grit bad wife Ther...
Name: review_text, Length: 5000, dtype: object

### Première tentative pour la partie 2

Nous vectorisons ensuite nos review textes pour cela nous utilisons le CountVectorizer celui-ci convertit des textes en vecteurs numériques en comptant les occurrences de chaque mot, créant ainsi une représentation matricielle. Puis nous utilisons trai_test_split comme vu en cours en effet celui-ci divise un ensemble de données en un ensemble d'entraînement et un ensemble de test, facilitant l'évaluation des performances des modèles d'apprentissage automatique en mesurant leur capacité à généraliser sur de nouvelles données. Nous utilisons cette technique pour tous nos classifier.

In [7]:
vectorizer = CountVectorizer()
reviewVectorised = vectorizer.fit_transform(DataForLearning['review_text'])
trainX, testX, trainY, testY = train_test_split(reviewVectorised,DataForLearning["label"] ,test_size=0.20)

Pour notre première tentative nous utilisions MultinomialNB en effet celui-ci est le plus approprié pour la classification de textes basée sur des comptages, comme la fréquence des mots.

In [8]:
classifier = MultinomialNB()
classifier.fit(trainX, trainY)

# Make predictions on the test set
predictions = classifier.predict(testX)

# Evaluate the model
accuracy = accuracy_score(testY, predictions)
report = classification_report(testY, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.878
Classification Report:
               precision    recall  f1-score   support

     negatif       0.91      0.86      0.88       530
     positif       0.85      0.90      0.87       470

    accuracy                           0.88      1000
   macro avg       0.88      0.88      0.88      1000
weighted avg       0.88      0.88      0.88      1000



Pour visualiser de façon plus concret nous avons fais le choix de tester avec des phrases qui ne sont PAS dans le dataset original pour tester la prédiction en condition réelle.

In [12]:
test_sentences = [
    "The service was slow, and the food was tasteless.",
    "Amazing atmosphere and a diverse menu.",
    "I would not recommend this place. The service was terrible.",
    "Delicious food and friendly staff.",
]


processed_test_sentences = [
    " ".join([token.lemma_ for token in nlp(sentence) if not token.is_stop]) for sentence in test_sentences
]

test_sentences_vectorized = vectorizer.transform(processed_test_sentences)
predictions_test = classifier.predict(test_sentences_vectorized)

for sentence, prediction in zip(test_sentences, predictions_test):
    print(f"Sentence: {sentence}")
    print(f"Predicted Sentiment: {prediction}\n")

Sentence: The service was slow, and the food was tasteless.
Predicted Sentiment: negatif

Sentence: Amazing atmosphere and a diverse menu.
Predicted Sentiment: positif

Sentence: I would not recommend this place. The service was terrible.
Predicted Sentiment: negatif

Sentence: Delicious food and friendly staff.
Predicted Sentiment: positif



Les tests sont plus que concluant mais pour réaliser la partie bonus nous avons fais le choix de tester avec deux classifier différents

### Alternative à la première tentative pour la partie 2

Pour cette première alternative nous avons fais le choix d'utiliser KNN (k plus proche voisin) pour appliquer un technique utiliser en cours.

In [11]:
#Le code en commentaire nous a permit de sélectionner le meilleur k pour le KNN
"""
tab_score_binary = []
for k in range(1,50):
  kloop= KNeighborsClassifier(k)
  tab_score_binary.append(np.mean(cross_val_score(kloop, trainX, trainY, cv=10)))
knn = np.mean(tab_score_binary)
"""
knn =1
knn_classifier = KNeighborsClassifier(n_neighbors= knn )  # You can adjust the number of neighbors (n_neighbors) as needed
knn_classifier.fit(trainX, trainY)

# Make predictions on the test set
predictions_knn = knn_classifier.predict(testX)

# Evaluate the k-NN model
accuracy_knn = accuracy_score(testY, predictions_knn)
report_knn = classification_report(testY, predictions_knn)

print("Accuracy (k-NN):", accuracy_knn)
print("matrice confusion = ", confusion_matrix(testY, predictions_knn))
print("Classification Report (k-NN):\n", report_knn)

Accuracy (k-NN): 0.633
matrice confusion =  [[222 308]
 [ 59 411]]
Classification Report (k-NN):
               precision    recall  f1-score   support

     negatif       0.79      0.42      0.55       530
     positif       0.57      0.87      0.69       470

    accuracy                           0.63      1000
   macro avg       0.68      0.65      0.62      1000
weighted avg       0.69      0.63      0.62      1000



In [14]:
test_sentences = [
    "The service was slow, and the food was tasteless.",
    "Amazing atmosphere and a diverse menu.",
    "I would not recommend this place. The service was terrible.",
    "Delicious food and friendly staff.",
]
processed_test_sentences = [
    " ".join([token.lemma_ for token in nlp(sentence) if not token.is_stop]) for sentence in test_sentences
]


test_sentences_vectorized = vectorizer.transform(processed_test_sentences)


predictions_test = knn_classifier.predict(test_sentences_vectorized)


for sentence, prediction in zip(test_sentences, predictions_test):
    print(f"Sentence: {sentence}")
    print(f"Predicted Sentiment: {prediction}\n")

Sentence: The service was slow, and the food was tasteless.
Predicted Sentiment: positif

Sentence: Amazing atmosphere and a diverse menu.
Predicted Sentiment: positif

Sentence: I would not recommend this place. The service was terrible.
Predicted Sentiment: positif

Sentence: Delicious food and friendly staff.
Predicted Sentiment: positif



Après analyse des résultats pour les mêmes phrases et le même jeu de données d'apprentissage le résultat est moins bon malgré un score d'accuracy d'environ 0.6-0.7

### Alternative 3 pour la partie 2 : Utilisation de SVC

In [15]:
vectorizer = CountVectorizer()
reviewVectorised = vectorizer.fit_transform(DataForLearning['review_text'])
trainX, testX, trainY, testY = train_test_split(reviewVectorised,DataForLearning["label"] ,test_size=0.20)

svm_classifier = SVC(kernel='linear') 
svm_classifier.fit(trainX, trainY)

predictions_svm = svm_classifier.predict(testX)
accuracy_svm = accuracy_score(testY, predictions_svm)
report_svm = classification_report(testY, predictions_svm)

print("Accuracy (SVM):", accuracy_svm)
print("Classification Report (SVM):\n", report_svm)

Accuracy (SVM): 0.881
Classification Report (SVM):
               precision    recall  f1-score   support

     negatif       0.90      0.87      0.88       523
     positif       0.86      0.90      0.88       477

    accuracy                           0.88      1000
   macro avg       0.88      0.88      0.88      1000
weighted avg       0.88      0.88      0.88      1000



In [24]:
test_sentences = [
    "The service was slow, and the food was tasteless.",
    "Amazing atmosphere and a diverse menu.",
    "I would not recommend this place. The service was terrible.",
    "Delicious food and friendly staff.",
]


processed_test_sentences = [
    " ".join([token.lemma_ for token in nlp(sentence) if not token.is_stop]) for sentence in test_sentences
]


test_sentences_vectorized = vectorizer.transform(processed_test_sentences)


predictions_test = svm_classifier.predict(test_sentences_vectorized)


for sentence, prediction in zip(test_sentences, predictions_test):
    print(f"Sentence: {sentence}")
    print(f"Predicted Sentiment: {prediction}\n")

Sentence: The service was slow, and the food was tasteless.
Predicted Sentiment: negatif

Sentence: Amazing atmosphere and a diverse menu.
Predicted Sentiment: positif

Sentence: I would not recommend this place. The service was terrible.
Predicted Sentiment: negatif

Sentence: Delicious food and friendly staff.
Predicted Sentiment: positif



Après analyse des résultats nous constatons un résultat plus que concluant mais il reste moins bon que le cas d'utilisation de MultinomialNB.

# Partie 3

Pour cette partie aussi nous allons utiliser trois classifier pour répondre au bonnus. \

### Première cas utilisation de MultinomialNB.

In [25]:
trainX, testX, trainY, testY = train_test_split(reviewVectorised,DataForLearning["rating"] ,test_size=0.20)

In [26]:
vectorizer = CountVectorizer()
reviewVectorised = vectorizer.fit_transform(DataForLearning['review_text'])
trainX, testX, trainY, testY = train_test_split(reviewVectorised,DataForLearning["rating"] ,test_size=0.20)
classifier = MultinomialNB()
classifier.fit(trainX, trainY)


predictions = classifier.predict(testX)


accuracy = accuracy_score(testY, predictions)
report = classification_report(testY, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.636
Classification Report:
               precision    recall  f1-score   support

           1       0.66      0.83      0.73       317
           2       0.41      0.38      0.39       202
           3       0.00      0.00      0.00        35
           4       0.16      0.06      0.09       100
           5       0.78      0.84      0.81       346

    accuracy                           0.64      1000
   macro avg       0.40      0.42      0.40      1000
weighted avg       0.58      0.64      0.60      1000



In [27]:
test_sentences = [
    "The service was slow, and the food was tasteless.",
    "Amazing atmosphere and a diverse menu.",
    "I would not recommend this place. The service was terrible.",
    "Delicious food and friendly staff.",
]


processed_test_sentences = [
    " ".join([token.lemma_ for token in nlp(sentence) if not token.is_stop]) for sentence in test_sentences
]

test_sentences_vectorized = vectorizer.transform(processed_test_sentences)
predictions_test = classifier.predict(test_sentences_vectorized)


for sentence, prediction in zip(test_sentences, predictions_test):
    print(f"Sentence: {sentence}")
    print(f"Predicted Sentiment: {prediction}\n")

Sentence: The service was slow, and the food was tasteless.
Predicted Sentiment: 1

Sentence: Amazing atmosphere and a diverse menu.
Predicted Sentiment: 5

Sentence: I would not recommend this place. The service was terrible.
Predicted Sentiment: 1

Sentence: Delicious food and friendly staff.
Predicted Sentiment: 5



Les tests pour ce classifier sont concluant cependant comme prévu la prediction pour notes dites neutres (3) est quasiment impossible.

### Alternative 2 à la partie 3 : Utilisation de la méthode KNN:

In [28]:
vectorizer = CountVectorizer()
reviewVectorised = vectorizer.fit_transform(DataForLearning['review_text'])
trainX, testX, trainY, testY = train_test_split(reviewVectorised,DataForLearning["rating"] ,test_size=0.20)

knn =1
knn_classifier = KNeighborsClassifier(n_neighbors= knn )  
knn_classifier.fit(trainX, trainY)


predictions_knn = knn_classifier.predict(testX)


accuracy_knn = accuracy_score(testY, predictions_knn)
report_knn = classification_report(testY, predictions_knn)

print("Accuracy (k-NN):", accuracy_knn)
print("matrice confusion = ", confusion_matrix(testY, predictions_knn))
print("Classification Report (k-NN):\n", report_knn)

Accuracy (k-NN): 0.435
matrice confusion =  [[103  64  13  23 109]
 [ 59  44  11  36  69]
 [  0   3   2   7  13]
 [  6   6   2  37  52]
 [ 15  11   2  64 249]]
Classification Report (k-NN):
               precision    recall  f1-score   support

           1       0.56      0.33      0.42       312
           2       0.34      0.20      0.25       219
           3       0.07      0.08      0.07        25
           4       0.22      0.36      0.27       103
           5       0.51      0.73      0.60       341

    accuracy                           0.43      1000
   macro avg       0.34      0.34      0.32      1000
weighted avg       0.45      0.43      0.42      1000



Comme lors de la partie 2 ce classifier ne répond pas aux attentes fixé pour l'exercice, avec une prédiction plus que peux fiable. Suite à ce manque de résultat concluant nous ne montrerons pas d'exemple.

### Alternative 3 pour la partie 3 : Utilisation de SVC Linear

In [29]:
vectorizer = CountVectorizer()
reviewVectorised = vectorizer.fit_transform(DataForLearning['review_text'])
trainX, testX, trainY, testY = train_test_split(reviewVectorised,DataForLearning["rating"] ,test_size=0.20)

svm_classifier = SVC(kernel='linear')
svm_classifier.fit(trainX, trainY)


predictions_svm = svm_classifier.predict(testX)


accuracy_svm = accuracy_score(testY, predictions_svm)
report_svm = classification_report(testY, predictions_svm)

print("Accuracy (SVM):", accuracy_svm)
print("Classification Report (SVM):\n", report_svm)

Accuracy (SVM): 0.604
Classification Report (SVM):
               precision    recall  f1-score   support

           1       0.62      0.66      0.64       314
           2       0.39      0.37      0.38       201
           3       0.04      0.05      0.04        19
           4       0.34      0.29      0.32       102
           5       0.81      0.80      0.80       364

    accuracy                           0.60      1000
   macro avg       0.44      0.44      0.44      1000
weighted avg       0.60      0.60      0.60      1000



Après analyse des résultats nous pouvons conclure que ce classifier est peut fiable mais reste notre second plus fiable avec en moyenne 3% de score d'accuracy de différence avec MultinomialNB.

# Partie 4 : Bonus

Cette partie à été raliser tout au long des parties 2 et 3. Via l'utilation de différent classifier.

A partir de la découmentation j'ai tenté de résumer au mieu comment fonctionne les différents classifier: \
 KNN fonctionne en trouvant les voisins les plus proches dans l'espace des caractéristiques pour prédire la classe d'un point, adapté à des données de dimensions moyennes à élevées. \
 SVC cherche un hyperplan optimal de séparation entre les classes, adapté aux problèmes de classification linéaire et non linéaire. \
 MultinomialNB, quant à lui, repose sur le théorème de Bayes et est particulièrement utile pour les données textuelles, modélisant la distribution multinomiale des caractéristiques